In [ ]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_features_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
labels_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

In [3]:
df_train = pd.read_csv(train_features_path, header=None)
df_labels = pd.read_csv(labels_path, header=None)

In [4]:
df_train.columns = [str(i) for i in range(df_train.shape[1])]
df_labels.columns = [str(i) for i in range(df_labels.shape[1])]

In [5]:
def apply_cwt_to_single_row(row, num_columns_per_channel, scales, waveletname='morl'):
    BHZ = row[:num_columns_per_channel]
    BHN = row[num_columns_per_channel:2*num_columns_per_channel]
    BHE = row[2*num_columns_per_channel:]
    
    cwt_BHZ, _ = pywt.cwt(BHZ, scales, waveletname)
    cwt_BHN, _ = pywt.cwt(BHN, scales, waveletname)
    cwt_BHE, _ = pywt.cwt(BHE, scales, waveletname)
    
    # Stack the CWT results for the 3 channels into a single array
    cwt_combined = np.stack([cwt_BHZ, cwt_BHN, cwt_BHE], axis=0)
    
    return cwt_combined

In [6]:
def apply_cwt_to_all_rows(data, num_columns_per_channel, scales, waveletname='morl'):
    cwt_results = []
    
    for _, row in data.iterrows():
        row_array = row.to_numpy()
        cwt_result = apply_cwt_to_single_row(row_array, num_columns_per_channel, scales, waveletname)
        cwt_results.append(cwt_result)
    
    return np.array(cwt_results)

In [8]:
scales = np.arange(1, 128) #Need to decide on the scales
num_columns_per_channel = 2401

cwt_transformed_data = apply_cwt_to_all_rows(df_train, num_columns_per_channel, scales)

MemoryError: Unable to allocate 111. GiB for an array with shape (16304, 3, 127, 2401) and data type float64

In [ ]:
df_train, df_test, train_labels, test_labels = train_test_split(cwt_transformed_data, df_labels, test_size=0.1, random_state=42, shuffle=True)